AlexNet主要贡献：

- 使用ReLu取代sigmoid，计算上更为简便
- 网络分布在多个GPU上，并且GPU之间只在少数时候沟通，从而提高效率（目前的GPU算力已经提升很多，分开构造不必要）
- Data Augmentation: 引入了大量的图像增广，如重采样、翻转、裁剪和颜色变化，从而进一步扩大数据集来缓解过拟合
- 使用Dropout，减少神经元之间的依赖性，从而减少过拟合

In [1]:
import sys
import time
import torch
from torch import nn, optim
import torchvision

import d2lzh_pytorch as d2l

# 设置GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

构造网络结构

In [2]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv = nn.Sequential(
            # 我们改用fashion-mnist，因此进入通道数为1，resize到224x224x1（保持和原论文一致）
            # in_channels, out_channels, kernel_size, stride, padding
            nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=2),  # 得到55x55x96
            nn.ReLU(),
            nn.MaxPool2d(3, 2),  # kernel_size, stride  (Overlapping Pooling，得到27x27x96)
            
            # 这个地方原论文看得我一脸懵逼，为什么原论文的通道数直接变成了256？
            # 为什么按照原论文使用256 kernels of size 5x5x48后结果不对？
            # 看Andrew Ng在他的CNN课程中才知道，原来这个地方使用了same convolution，所以是有padding的，但原论文没有提到
            
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),  # 得到27x27x256
            nn.ReLU(),
            nn.MaxPool2d(3, 2),  # 得到13x13x256
            
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),  # 又是same convolution，得到13x13x384
            nn.ReLU(),
            
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1), # same convolution，13x13x384
            nn.ReLU(),
            
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1), # same convolution, 13x13x256
            nn.ReLU(),
            nn.MaxPool2d(3, 2)  # 6x6x256
        )
        
        self.fc = nn.Sequential(
            # @ShusenTang 的代码中，in_features=5x5x256，与原论文不同，原因是没有在第一个conv2d加入padding
            nn.Linear(6*6*256, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),  # 在两个全连接层中采用丢弃法
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 10)  # softmax运算
        )
        

    def forward(self, img): 
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output

AlexNet的实例化，print查看网络结构

In [3]:
net = AlexNet()
print(net)

AlexNet(
  (conv): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=9216, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inpl

重写读入数据集的函数，加入resize参数，使用torchvision.transforms.Resize实现

In [4]:
def load_data_fashion_mnist(batch_size, resize=None, root='~/Datasets/FashionMNIST'):
    trans = []
    if resize:
        trans.append(torchvision.transforms.Resize(size=resize))
    trans.append(torchvision.transforms.ToTensor())
    
    transform = torchvision.transforms.Compose(trans)
    
    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True, download=True, transform=transform)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False, download=True, transform=transform)
    
    if sys.platform.startswith('win'):
        num_workers = 0  # 0表示不用额外的进程来加速读取数据
    else:
        num_workers = 4

    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    
    return train_iter, test_iter

In [5]:
batch_size = 128

train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)

训练模型（有点慢，但效果不错）

In [6]:
lr, num_epochs = 0.001, 5

optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 0.6067, train acc 0.768, test acc 0.866, time 136.4 sec
epoch 2, loss 0.3328, train acc 0.877, test acc 0.878, time 138.5 sec
epoch 3, loss 0.2905, train acc 0.892, test acc 0.885, time 139.6 sec
epoch 4, loss 0.2630, train acc 0.903, test acc 0.899, time 140.1 sec
epoch 5, loss 0.2384, train acc 0.911, test acc 0.903, time 140.3 sec


遗留问题：

- 原论文加入了0.0005的weight decay并且认为对于降低training error有帮助，可待验证（尽管我们目前用的优化器与原论文不同）
- 原论文第2、第4、第5卷积层的bias初始化为1，为了给ReLU提供positive input，现在还有没有这样初始化的必要呢（nn.Conv2d的bias初始化默认是一个y轴对称的均匀分布）